In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [2]:
# numpy, pandas, and regex libraries for manipulating data 
import re
import numpy as np 
import pandas as pd

# sklearn tools for ensemble models 
from sklearn.ensemble import RandomForestClassifier, forest
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score
from sklearn.tree import export_graphviz

## data viz libraries 
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import graphviz

In [5]:
print('https://www.kaggle.com/c/forest-cover-type-kernels-only')

https://www.kaggle.com/c/forest-cover-type-kernels-only


In [3]:
df_raw = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [4]:
df_raw.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
display_all(df_raw.tail().T)

In [ ]:
print(df_raw.shape)
display_all(df_raw.describe().T)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_raw.drop('Cover_Type',axis=1),
                                                    df_raw['Cover_Type'], 
                                                    test_size=0.20, 
                                                    random_state=42)
m = RandomForestClassifier(n_jobs=-1)
%time m.fit(X_train, y_train)

In [ ]:
# what is the accuracy? 
print('Train Accuracy:  '+str(m.score(X_train, y_train)))
print('Test Accuracy:   '+str(m.score(X_test, y_test)))

In [ ]:
print(classification_report(m.predict(X_test), y_test))

In [ ]:
mat = confusion_matrix(y_test, m.predict(X_test))
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

Female cones are up to 2 inches long and many remain closed and attached to the tree for numerous years. The cones are often tightly sealed with resin and woody tissue that only opens in extreme heat, typically from a fire. Hundreds of seeds will germinate after a fire and form an even-aged lodgepole forest. With such a dense canopy, future lodgepole seedlings are prohibited from growing because lodgepole pines don't tolerate shade. If left undisturbed, forest succession will play out in these lodgepole pine communities. Shade-tolerant spruce, subalpine fir or Douglas fir will grow in the understory of the lodgepole pine and eventually take over and become the dominant trees in the area.

**3. Ponderosa Pine**

Scientific name: _Pinus ponderosa_

Habitat: 5600 ft. to 9500 ft. (1680 m.-2850 m.), primarily the montane ecosystem of the park


**6. Douglas-Fir**

Scientific name: _Pseudotsuga menziesii_

Habitat: 5500 ft. - 11500 ft. (1650 m. - 3450 m.). Douglas-fir form dense dark forests on north facing montane slopes. Higher up in the subalpine it's found mixed into the forest on warmer south facing slopes.

## Single Tree 

In [ ]:
m = RandomForestClassifier(n_estimators=1, bootstrap=False, max_depth=3,n_jobs=-1)
m.fit(X_train, y_train)

In [ ]:
def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    """ Draws a representation of a random forest in IPython.

    Parameters:
    -----------
    t: The tree you wish to draw
    df: The data used to train the tree. This is used to get the names of the features.
    """
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

In [ ]:
draw_tree(m.estimators_[0], X_train, precision=3)

In [ ]:
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
m.fit(X_train, y_train)
print(m.score(X_train, y_train))
print(m.score(X_test, y_test))

## Bootstrap aggregation i.e. "Bagging"

In [ ]:
m = RandomForestClassifier(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print(m.score(X_train, y_train))
print(m.score(X_test, y_test))


In [ ]:
# show the predictions of each individual tree 
preds = np.stack([t.predict(X_test) for t in m.estimators_])+1 #python is zero-indexed, convert idx to cat code

In [ ]:
preds.shape

In [ ]:
preds[:,2]

In [ ]:
y_test.iloc[2]

In [ ]:
preds[:1]

In [ ]:

## need to implement a mode function to get the max vote of ensemble for each row 

plt.plot([roc_auc_score(pd.get_dummies(y_test[:i+1]), np.mode(preds[:i+1])) for i in range(10)]);

### Out-of-bag (OOB) score
Is our validation set worse than our training set because we're over-fitting, or because the validation set is for a different time period, or a bit of both? With the existing information we've shown, we can't tell. However, random forests have a very clever trick called out-of-bag (OOB) error which can handle this (and more!)

The idea is to calculate error on the training set, but only include the trees in the calculation of a row's error where that row was not included in training that tree. This allows us to see whether the model is over-fitting, without needing a separate validation set.

This also has the benefit of allowing us to see whether our model generalizes, even if we only have a small amount of data so want to avoid separating some out to create a validation set.

This is as simple as adding one more parameter to our model constructor. We print the OOB error last in our print_score function below.

In [ ]:
# test the effect of adding more trees to the forest 
m = RandomForestClassifier(n_estimators=200, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print(m.score(X_train, y_train))
print(m.score(X_test, y_test))
print(m.oob_score_)

## Reducing Overfitting

### Subsampling

The basic idea is this: rather than limit the total amount of data that our model can access, let's instead limit it to a *different* random subset per tree. That way, given enough trees, the model can still see *all* the data, but for each individual tree it'll be just as fast as if we had cut down our dataset as before.

In [ ]:
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

In [ ]:
set_rf_samples(5000)

In [ ]:
m = RandomForestClassifier(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print(m.score(X_train, y_train))
print(m.score(X_test, y_test))
print(m.oob_score_)

In [ ]:
reset_rf_samples()

## Feature Importance

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
fi = rf_feat_importance(m, X_train); fi[:10]

In [ ]:
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [ ]:
plot_fi(fi[:30]);

In [ ]:
sns.jointplot(x="Id", y="Elevation", kind="hex",data=X_train);

In [ ]:
sns.jointplot(x="Id", y="Horizontal_Distance_To_Roadways",kind='hex',data=X_train);